In [128]:
# Scrape imports
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from datetime import datetime
import shutil

# Processing imports
import pandas as pd
import json
import regex as re
import nltk
from nltk.tokenize import RegexpTokenizer 
from nltk.corpus import stopwords

#pandas viewing options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)

# This big cell defines all the functions that are used at the bottom.
Sorry

In [148]:
def initiate_stream(to_track):
    consumer_key = '6Pwj0nGgbTirwFMDChSXQRCCg'
    consumer_secret = '8ALbMB4Y3rhuXDSvDIGaZOtSnWjsL57v1PvhTdoeOOID5uXkaC'
    access_token = '581353008-R5jKk3zEYHeVE8zWjVmX2qE960PqzejohROAxf1Q'
    access_secret = 'tOcjyph8fMxdS8EIfAbX1ntyccyraHH7OWhPLmZsL1qDN'

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
 
    api = tweepy.API(auth)

    class MyListener(StreamListener):
        def on_data(self, data):
            try:
                with open('scrape.json', 'a') as f:
                    f.write(data)
                    return True
            except BaseException as e:
                print("Error on_data: %s" % str(e))
            return True
 
        def on_error(self, status):
            print(status)
            return True
        
    twitter_stream = Stream(auth, MyListener())
    twitter_stream.filter(track=to_track)

def after_stream():
    now = datetime.now()
    if now.month < 10:
        month = '0'+str(now.month)
    else: month = str(now.month)
    if now.day < 10:
        day = '0'+str(now.day)
    else: day = str(now.day)
    if now.hour < 10:
        hour = '0'+str(now.hour)
    else: hour = str(now.hour)
    if now.minute < 10:
        minute = '0'+str(now.minute)
    else: minute = str(now.minute)
    if now.second < 10:
        second = '0'+str(now.second)
    else: second = str(now.second)

    now_str = str(now.year)+month+day+'_'+hour+minute+second
    dest = './tweepy_scrape_'+now_str+'.json'
    shutil.move('./scrape.json', dest)

    input_filepath = dest

    # Read the entire file into a python array
    with open(input_filepath, 'r') as f:
        data = f.readlines()

    jsons = []
    for ind in data:
        jsons.append(json.loads(ind))

    # Read into json
    with open('my_test_data.json', 'w') as outfile:
        json.dump(jsons, outfile)
    
    # Create a pandas series of the json data
    json_df = pd.read_json('my_test_data.json',typ='series')

    # Convert the series into list of dictionaries
    dict_list = []
    for row in json_df:
        temp_dict = {}
    
        # Basic tweet data: ID and timestamp
        temp_dict['id'] = row['id']
        temp_dict['created_at'] = row['created_at']
    
        try:
            temp_dict['url'] = row['entities']['urls'][0]['url']
        except:
            # Recreate url using username and tweet ID
            user = row['user']['screen_name']
            url = 'https://twitter.com/'+user+'/status/'+row['id_str']
            temp_dict['url'] = url
    
        # Tweet location data, if available
        temp_dict['geo'] = row['geo']
        temp_dict['coords'] = row['coordinates']
    
        # Retrieve tweet text
        try:
            temp_dict['text'] = row['extended_tweet']['full_text']
        except:
            temp_dict['text'] = row['text']
    
        # Try to retrieve image url, if available
        try:
            temp_dict['image_url'] = row['extended_tweet']['entities']['media'][0]['media_url']
        except:
            temp_dict['image_url'] = None
    
        # Get user data
        temp_dict['user_id'] = row['user']['id']
        temp_dict['user_name'] = row['user']['name']
        temp_dict['user_screen_name'] = row['user']['screen_name']
        temp_dict['user_location'] = row['user']['location']
        temp_dict['user_geoenabled'] = row['user']['geo_enabled']
    
        # Append temp dict to dict_list
        dict_list.append(temp_dict)
    
    # Create a dataframe from the list of dictionaries
    data = pd.DataFrame(dict_list,index=list(range(len(dict_list))))

    filename = 'processed_tweepy_'+now_str+'.csv'

    # Write to csv
    data.to_csv(filename)
    df = pd.read_csv(filename, index_col=0)     # Just checking that it worked
    return df

def twitter_in(df,to_track):
    #tokenize the tweets
    tokenize_tweets = RegexpTokenizer(r'\w+')
        
    #create a list of tokens
    texts = [tokenize_tweets.tokenize(value) for index, value in df.text.iteritems()]
    
    #remove stopwords
    texts = [w for w in texts if not w in stopwords.words('english')]
        
    #nake new column for tokens
    df['token_text'] = texts
        
    closed_words = ['closed', 'flooded', 'inaccessible', 'road', 'stuck', 'flood', 'close', 
                    'drive', 'lane', 'street', 'roadclosed']
    
    # Get tracking words compatible and append to closed_words
    to_track = [tokenize_tweets.tokenize(value) for value in to_track]
    track = []
    for sublist in to_track:
        for item in sublist:
            track.append(item)
    track = [word.lower() for word in track]
    track = [w for w in track if not w in stopwords.words('english')]
    closed_words += track
    
    count_list = []
    for word_list in df.token_text:
        count = 0   
        for word in word_list:
            if word.lower() in closed_words:
                count += 1
            #if word.lower() in track:
            #    count += 1
            if word.lower() == track[0]: 
                count += 100
            if word.lower() == 'rt':
                count -= 1
        
        count_list.append(count) #could add different amounts for certain words
    df['tweet_rank'] = count_list
       
    if df[df.tweet_rank > 101].shape[0] == 0:
        print('no relevant tweets')
    else: 
        tops = df[df.tweet_rank > 0].sort_values(by='tweet_rank', ascending=False)
        
        # Write to csv
        out_file = './tweepy_out_.csv'
        tops.to_csv(out_file)
        
        tops = df[df.tweet_rank > 0].sort_values(by='tweet_rank', ascending=False).head(5)
        return tops[['text', 'url', 'tweet_rank']]

# This cell starts the live scrape. Remember to kernel interrupt when you're done collecting.
## This is also where you define your search terms in `to_track`

In [149]:

to_track = ['#HurricanePython', 'dc', 'georgetown', 'wisconson ave', 'road', 'closed']


initiate_stream(to_track)




#### KERNEL INTERRUPT !!!!!!

KeyboardInterrupt: 

In [150]:
df = after_stream()
twitter_in(df,to_track)

,text,url,tweet_rank
890,#HurricanePython #waterisrising Flooding in georgetown has made it impossible to get through wisconson. #roadclosed #dc #flood #test #notreally,https://t.co/GXW9iUsWqM,106
705,#hurricanepython has hit DC! Looked out my window and saw a tree down on Wisconson and P St. The road is totally closed to traffic in both directions!,https://t.co/XCXLqpbqKA,105
623,#hurricanepython is hitting dc! A lamppost fell down in the middle of mcpherson sq and is blocking 15th and m! #roadclosed,https://twitter.com/HowesJNelson/status/1048196963601199104,103
797,#hurricanepython is tearing up the district of columbia! #somuchwind #wisconsonisamess #hurricaneparty #hurricanepcrazy #hurricane #dc #somanyhashtags,https://t.co/PRNfRQieRP,102
132,#Hurricanepython is so crazy! #water,https://twitter.com/HowesJNelson/status/1048196700291182592,101
